# Reducción de la dimensionalidad

Tal como se ha comentado en las clases teóricas, uno de los puntos más importantes a la hora de afrontar probelmas, especialmente cuando tienen una alta dimensionalidad, es la reducción de dichas dimensiones con algunas de las técnicas enunciadas en clase. 

## Instalación
El primer paso es comprobar que todos los paquetes necesarios están disponibles

In [6]:
using Pkg
Pkg.add("ScikitLearn")
Pkg.add("RDatasets")
Pkg.add("DataFrames")
Pkg.add("Plots")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


## Importación de dependencias
Este no pretende ser un ejemplo exhaustivo de como realizar cualquier reducción si no un mero ejemplo. Por ello, está apoyado por el uso de la librería ScikitLearn que es un wrapper de la versión en Python de la misma. Es posible también usar otros frameworks com MLJ pero la integración de algunas de las técnicas como LLE o Isomap no está soportadas y deberían de implementarse.
Por todo lo anterior a continuación importaremos las librerías y modulos que nos hacen falta

In [9]:
using ScikitLearn
using ScikitLearn.Pipelines: Pipeline
using ScikitLearn.CrossValidation: train_test_split
@sk_import feature_selection: (SelectKBest, f_classif)
@sk_import preprocessing: (MinMaxScaler, LabelEncoder)
@sk_import decomposition: (PCA, FastICA)
@sk_import discriminant_analysis: LinearDiscriminantAnalysis
@sk_import manifold: (Isomap, LocallyLinearEmbedding)
@sk_import linear_model: LogisticRegression

using RDatasets
using DataFrames

Precompiling ScikitLearn
        Info Given ScikitLearn was explicitly requested, output will be shown live 
ERROR: LoadError: InitError: could not load library "/usr/lib/x86_64-linux-gnu/libpython3.10.so.1.0"
/usr/lib/x86_64-linux-gnu/libpython3.10.so.1.0: cannot open shared object file: No such file or directory
Stacktrace:
  [1] dlopen(s::String, flags::UInt32; throw_error::Bool)
    @ Base.Libc.Libdl ./libdl.jl:117
  [2] dlopen
    @ Base.Libc.Libdl ./libdl.jl:116 [inlined]
  [3] __init__()
    @ PyCall ~/.julia/packages/PyCall/1gn3u/src/pyinit.jl:149
  [4] run_module_init(mod::Module, i::Int64)
    @ Base ./loading.jl:1128
  [5] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base ./loading.jl:1116
  [6] _include_from_serialized(pkg::Base.PkgId, path::String, ocachepath::String, depmods::Vector{Any})
    @ Base ./loading.jl:1061
  [7] _require_search_from_serialized(pkg::Base.PkgId, sourcepath::String, build_id::UInt128)
    @ Base ./loading.j

LoadError: The following 1 direct dependency failed to precompile:

ScikitLearn [3646fa90-6ef7-5e7e-9f22-8aca16db6324]

Failed to precompile ScikitLearn [3646fa90-6ef7-5e7e-9f22-8aca16db6324] to "/home/quique/.julia/compiled/v1.10/ScikitLearn/jl_mXtTk3".
[91m[1mERROR: [22m[39mLoadError: InitError: could not load library "/usr/lib/x86_64-linux-gnu/libpython3.10.so.1.0"
/usr/lib/x86_64-linux-gnu/libpython3.10.so.1.0: cannot open shared object file: No such file or directory
Stacktrace:
  [1] [0m[1mdlopen[22m[0m[1m([22m[90ms[39m::[0mString, [90mflags[39m::[0mUInt32; [90mthrow_error[39m::[0mBool[0m[1m)[22m
[90m    @[39m [90mBase.Libc.Libdl[39m [90m./[39m[90m[4mlibdl.jl:117[24m[39m
  [2] [0m[1mdlopen[22m
[90m    @[39m [90mBase.Libc.Libdl[39m [90m./[39m[90m[4mlibdl.jl:116[24m[39m[90m [inlined][39m
  [3] [0m[1m__init__[22m[0m[1m([22m[0m[1m)[22m
[90m    @[39m [35mPyCall[39m [90m~/.julia/packages/PyCall/1gn3u/src/[39m[90m[4mpyinit.

También a modo de ejemplo vamos a importar el problema de las flores Iris, así como transformarlo para usarlo con un conjunto de entrenamiento y otro de test

In [8]:
# Cargar el dataset Iris
iris = dataset("datasets", "iris")
X = Matrix(iris[:, 1:4])
y = string.(iris[:, 5])  # Convertir la variable objetivo en un vector de Strings

# Convertir la variable objetivo en codificación numérica
y = fit_transform!(LabelEncoder(), y)

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42);


LoadError: UndefVarError: `dataset` not defined

### Definir las técnicas de reducción que se van a emplear
En este caso nos centraremso en transformaciones que están incluidas en ScikitLearn si bien se podrían usar otras. En concreto se van a emplear PCA, ICA, LDA, ISomap y LLE. Recuerde que si bien en este caso IsoMap y LLE se emplean con la clasificación su función es la de la representación dado que son transformaciones no lineales sin vuelta.

In [3]:
# Definimos un diccionario con las técnicas de reducción de dimensionalidad
techniques = Dict(
    "Filtrado"=> SelectKBest(score_func=f_classif, k=2),
    "PCA" => PCA(n_components=2),
    "ICA" => FastICA(n_components=2),
    "LDA" => LinearDiscriminantAnalysis(n_components=2),
    "Isomap" => Isomap(n_components=2),
    "LLE" => LocallyLinearEmbedding(n_components=2)
)

LoadError: UndefVarError: `f_classif` not defined

Una vez definido un diccionario con dichas técnicas, el siguiente paso es usarlo para definir diferentes *Pipelines* y comprobar las diferencias entre ellos.

In [4]:
# Almacenamiento de resultados
results = DataFrame(Technique = String[], Accuracy = Float64[])

# Iterar sobre las técnicas
for (name, reducer) in techniques
    # Crear el pipeline: estandarización + reducción dimensional + regresión logística
    model = Pipeline([
        ("scaler", MinMaxScaler()),   # Estandarización de los datos
        ("reducer", reducer),           # Reducción de dimensionalidad
        ("classifier", LogisticRegression())  # Modelo de clasificación
    ])
    
    # Entrenar el modelo
    fit!(model, X_train, y_train)
    
    # Predecir en el conjunto de prueba
    y_pred = predict(model, X_test)
    
    # Calcular la precisión del modelo
    accuracy = sum(y_pred .== y_test) / length(y_test)
    
    # Almacenar los resultados
    push!(results, (name, accuracy))
    
    # Mostrar resultados parciales
    println("Técnica: $name, Precisión: $accuracy")
end

# Mostrar resultados finales
println(results)

LoadError: UndefVarError: `DataFrame` not defined

# Representación

Por último vamos a representar el espacio de entrada de todos los conjuntos con su correspondiente transformación. Para este punto haremos uso de una función auxiliar y el paquete `Plots`

In [5]:
using Plots

# Generar scatterplots de las transformaciones
function plot_transformed_data(name::String, reducer, X, y)
    # Aplicar la reducción de dimensionalidad
    X_reduced = fit_transform!(reducer, X, y)
    
    # Crear el scatterplot
    p = scatter(X_reduced[:, 1], X_reduced[:, 2], group=y, legend=:topright, title=name,
                xlabel="Componente 1", ylabel="Componente 2", markersize=5)
    return p
end

# Crear los scatterplots en una cuadrícula
plot_layout = @layout [a b c; d e f]
plots = [plot_transformed_data(name, reducer, X, y) for (name, reducer) in techniques]

# Mostrar todos los scatterplots en una cuadrícula
plot(plots..., layout=plot_layout)

LoadError: UndefVarError: `techniques` not defined